# Conversion from top predictive model to pipelines

I'll be using this script and seeing how easy it is to convert to the pipeline methodology.

https://www.kaggle.com/aharless/xgboost-using-4th-quarter-for-validation/notebook

## Libraries

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import datetime as dt
from datetime import datetime
import gc
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg

C:\Users\Evan\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Evan\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Setting up the run parameters

In [3]:
runDetails = dict()
modelDesc = dict()

runDetails['MAKE_SUBMISSION'] = True          # Generate output file.
runDetails['CV_ONLY'] = False                 # Do validation only; do not generate predicitons.
runDetails['FIT_FULL_TRAIN_SET'] = True       # Fit model to full training set after doing validation.
runDetails['FIT_2017_TRAIN_SET'] = False      # Use 2017 training data for full fit (no leak correction)
runDetails['USE_SEASONAL_FEATURES'] = True
runDetails['VAL_SPLIT_DATE'] = '2016-09-15'   # Cutoff date for validation split
runDetails['FUDGE_FACTOR_SCALEDOWN'] = 0.3    # exponent to reduce optimized fudge factor for prediction
runDetails['OPTIMIZE_FUDGE_FACTOR'] = True    # Optimize factor by which to multiply predictions.

modelDesc['LEARNING_RATE'] = 0.007           # shrinkage rate for boosting roudns
modelDesc['ROUNDS_PER_ETA'] = 20             # maximum number of boosting rounds times learning rate

## Reading in data and encoding objects to int

In [29]:
import os

os.chdir('C:/Users/Evan/Documents/GitHub/Zillow/data')

properties = pd.read_csv('properties_2016.csv', low_memory = False)
properties17 = pd.read_csv('properties_2017.csv', low_memory = False)

train = pd.read_csv("train_2016_v2.csv", low_memory = False)

for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

In [28]:
train_df = train.merge(properties, how='left', on='parcelid')
select_qtr4 = pd.to_datetime(train_df["transactiondate"]) >= runDetails['VAL_SPLIT_DATE']
if runDetails['USE_SEASONAL_FEATURES']:
    basedate = pd.to_datetime('2015-11-15').toordinal()

NameError: name 'train' is not defined

In [6]:
del train
gc.collect()

160

## Creating agg features

In [7]:
# Inputs to features that depend on target variable
# (Ideally these should be recalculated, and the dependent features recalculated,
#  when fitting to the full training set.  But I haven't implemented that yet.)

# Standard deviation of target value for properties in the city/zip/neighborhood
citystd = train_df[~select_qtr4].groupby('regionidcity')['logerror'].aggregate("std").to_dict()
zipstd = train_df[~select_qtr4].groupby('regionidzip')['logerror'].aggregate("std").to_dict()
hoodstd = train_df[~select_qtr4].groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()

In [27]:
def calculate_features(df):
    
    # Nikunj's features
    # Number of properties in the zip
    df['N-zip_count'] = df['regionidzip'].agg('count')
    # Number of properties in the city
    df['N-city_count'] = df['regionidcity'].agg('count')
    # Does property have a garage, pool or hot tub and AC?
    df['N-GarPoolAC'] = ((df['garagecarcnt']>0) & \
                         (df['pooltypeid10']>0) & \
                         (df['airconditioningtypeid']!=5))*1 

    # More features
    # Mean square feet of neighborhood properties
    df['mean_area'] = df.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].agg('median')
    # Median year of construction of neighborhood properties
    df['med_year'] = df.groupby('regionidneighborhood')['yearbuilt'].agg('median')
    # Neighborhood latitude and longitude
    df['med_lat'] = df.groupby('regionidneighborhood')['latitude'].agg('median')
    df['med_long'] = df.groupby('regionidneighborhood')['longitude'].agg('median')

    df['zip_std'] = df['regionidzip'].map(zipstd)
    df['city_std'] = df['regionidcity'].map(citystd)
    df['hood_std'] = df['regionidneighborhood'].map(hoodstd)
    
    if runDetails['USE_SEASONAL_FEATURES']:
        df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
        df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)  
        
    return(df)

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class FeatureSummariser(BaseEstimator, TransformerMixin):
    
    def __init__(self, subset, agg):
        
        self.subset = subset
        self.agg = agg

    def transform(self, X, *_):
        
        
        
        return X * self.factor

    def fit(self, *_):
        return self

In [25]:
dropvars = ['parcelid', 'airconditioningtypeid', 'buildingclasstypeid',
            'buildingqualitytypeid', 'regionidcity']
droptrain = ['logerror', 'transactiondate']

In [26]:
train_df = calculate_features(train_df)

x_valid = train_df.drop(dropvars+droptrain, axis=1)[select_qtr4]
y_valid = train_df["logerror"].values.astype(np.float32)[select_qtr4]

print('Shape full training set: {}'.format(train_df.shape))
print('Dropped vars: {}'.format(len(dropvars+droptrain)))
print('Shape valid X: {}'.format(x_valid.shape))
print('Shape valid y: {}'.format(y_valid.shape))

train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
print('\nFull training set after removing outliers, before dropping vars:')     
print('Shape training set: {}\n'.format(train_df.shape))

if runDetails['FIT_FULL_TRAIN_SET']:
    full_train = train_df.copy()

train_df=train_df[~select_qtr4]
x_train=train_df.drop(dropvars+droptrain, axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)
n_train = x_train.shape[0]
print('Training subset after removing outliers:')     
print('Shape train X: {}'.format(x_train.shape))
print('Shape train y: {}'.format(y_train.shape))

if runDetails['FIT_FULL_TRAIN_SET']:
    x_full = full_train.drop(dropvars+droptrain, axis=1)
    y_full = full_train["logerror"].values.astype(np.float32)
    n_full = x_full.shape[0]
    print('\nFull trainng set:')     
    print('Shape train X: {}'.format(x_train.shape))
    print('Shape train y: {}'.format(y_train.shape))

NameError: name 'train_df' is not defined

In [11]:
if not runDetails['CV_ONLY']:
    test_df = properties.copy()
    droptest = []
    if runDetails['USE_SEASONAL_FEATURES']:
        test_df['transactiondate'] = '2016-10-31'
        droptest = ['transactiondate']
    calculate_features(test_df)
    x_test = test_df.drop(dropvars+droptest, axis=1)
    print('Shape test: {}'.format(x_test.shape))

    # Process properties for 2017
    properties = properties17
    for c in properties.columns:
        properties[c]=properties[c].fillna(-1)
        if properties[c].dtype == 'object':
            lbl = LabelEncoder()
            lbl.fit(list(properties[c].values))
            properties[c] = lbl.transform(list(properties[c].values))

    test_df = properties.copy()
    if runDetails['USE_SEASONAL_FEATURES']:
        test_df['transactiondate'] = '2017-10-31'
    test_df = calculate_features(test_df)
    x_test17 = test_df.drop(dropvars+droptest, axis=1)   
    del test_df

Shape test: (2985217, 65)


In [12]:
del train_df
del select_qtr4
gc.collect()

28

In [13]:
xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': modelDesc['LEARNING_RATE'],
    'max_depth': 7, 
    'subsample': 0.6,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 5.0,
    'alpha': 0.65,
    'colsample_bytree': 0.5,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

## Getting the data into a xgboost friendly format

In [14]:
dtrain = xgb.DMatrix(x_train, y_train)
dvalid_x = xgb.DMatrix(x_valid)
dvalid_xy = xgb.DMatrix(x_valid, y_valid)

if not runDetails['CV_ONLY']:
    dtest = xgb.DMatrix(x_test)
    dtest17 = xgb.DMatrix(x_test17)
    del x_test

In [15]:
del x_train
gc.collect()

181

In [16]:
num_boost_rounds = round( modelDesc['ROUNDS_PER_ETA'] / xgb_params['eta'] )
early_stopping_rounds = round( num_boost_rounds / 20 )
print('Boosting rounds: {}'.format(num_boost_rounds))
print('Early stoping rounds: {}'.format(early_stopping_rounds))

Boosting rounds: 2857
Early stoping rounds: 143


In [17]:
evals = [(dtrain,'train'),(dvalid_xy,'eval')]
model = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds,
                  evals=evals, early_stopping_rounds=early_stopping_rounds, 
                  verbose_eval=10)

[0]	train-mae:0.053445	eval-mae:0.065272
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 143 rounds.
[10]	train-mae:0.053359	eval-mae:0.065206
[20]	train-mae:0.053273	eval-mae:0.065138
[30]	train-mae:0.053193	eval-mae:0.065084
[40]	train-mae:0.053112	eval-mae:0.065028
[50]	train-mae:0.053035	eval-mae:0.064978
[60]	train-mae:0.05296	eval-mae:0.064928
[70]	train-mae:0.052894	eval-mae:0.064886
[80]	train-mae:0.052834	eval-mae:0.064848
[90]	train-mae:0.052778	eval-mae:0.064813
[100]	train-mae:0.05273	eval-mae:0.064784
[110]	train-mae:0.052672	eval-mae:0.064748
[120]	train-mae:0.052624	eval-mae:0.06472
[130]	train-mae:0.052578	eval-mae:0.064699
[140]	train-mae:0.052534	eval-mae:0.064676
[150]	train-mae:0.052488	eval-mae:0.064649
[160]	train-mae:0.052442	eval-mae:0.064626
[170]	train-mae:0.052396	eval-mae:0.064605
[180]	train-mae:0.052357	eval-mae:0.064585
[190]	train-mae:0.052323	eval-mae:0.064568
[200]	train-

In [18]:
valid_pred = model.predict(dvalid_x, ntree_limit=model.best_ntree_limit)
print( "XGBoost validation set predictions:" )
print( pd.DataFrame(valid_pred).head() )
print("\nMean absolute validation error:")
mean_absolute_error(y_valid, valid_pred)

XGBoost validation set predictions:
          0
0  0.001453
1  0.022785
2  0.020948
3  0.012848
4  0.018639

Mean absolute validation error:


0.064313427

In [19]:
if runDetails['OPTIMIZE_FUDGE_FACTOR']:
    mod = QuantReg(y_valid, valid_pred)
    res = mod.fit(q=.5)
    print("\nLAD Fit for Fudge Factor:")
    print(res.summary())

    fudge = res.params[0]
    print("Optimized fudge factor:", fudge)
    print("\nMean absolute validation error with optimized fudge factor: ")
    print(mean_absolute_error(y_valid, fudge*valid_pred))

    fudge **= runDetails['FUDGE_FACTOR_SCALEDOWN']
    print("Scaled down fudge factor:", fudge)
    print("\nMean absolute validation error with scaled down fudge factor: ")
    print(mean_absolute_error(y_valid, fudge*valid_pred))
else:
    fudge=1.0


LAD Fit for Fudge Factor:
                         QuantReg Regression Results                          
Dep. Variable:                      y   Pseudo R-squared:              0.01241
Model:                       QuantReg   Bandwidth:                    0.009326
Method:                 Least Squares   Sparsity:                       0.1035
Date:                Sun, 08 Oct 2017   No. Observations:                14304
Time:                        11:58:21   Df Residuals:                    14303
                                        Df Model:                            1
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0030      0.029     34.898      0.000       0.947       1.059
Optimized fudge factor: 1.00304

Mean absolute validation error with optimized fudge factor: 
0.0643134
Scaled down fudge factor: 1.00091240994

Mean absolute validation error with sc

In [20]:
if runDetails['FIT_FULL_TRAIN_SET'] and not runDetails['CV_ONLY']:
    if runDetails['FIT_2017_TRAIN_SET']:
        
        train = pd.read_csv('train_2017.csv')
        train_df = train.merge(properties, how='left', on='parcelid')
        citystd = train_df.groupby('regionidcity')['logerror'].aggregate("std").to_dict()
        zipstd = train_df.groupby('regionidzip')['logerror'].aggregate("std").to_dict()
        hoodstd = train_df.groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()
        train_df = calculate_features(train_df)
        x_full = train_df.drop(dropvars+droptrain, axis=1)
        y_full = train_df["logerror"].values.astype(np.float32)
        n_full = x_full.shape[0]     
        
    dtrain = xgb.DMatrix(x_full, y_full)
    num_boost_rounds = int(model.best_ntree_limit*n_full/n_train)
    full_model = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds, 
                           evals=[(dtrain,'train')], verbose_eval=10)

[0]	train-mae:0.053266
[10]	train-mae:0.053178
[20]	train-mae:0.053096
[30]	train-mae:0.053014
[40]	train-mae:0.05294
[50]	train-mae:0.052866
[60]	train-mae:0.0528
[70]	train-mae:0.052739
[80]	train-mae:0.052684
[90]	train-mae:0.05263
[100]	train-mae:0.052579
[110]	train-mae:0.052528
[120]	train-mae:0.05248
[130]	train-mae:0.052433
[140]	train-mae:0.052386
[150]	train-mae:0.05234
[160]	train-mae:0.0523
[170]	train-mae:0.052257
[180]	train-mae:0.052221
[190]	train-mae:0.052186
[200]	train-mae:0.052151
[210]	train-mae:0.052119
[220]	train-mae:0.052082
[230]	train-mae:0.05205
[240]	train-mae:0.052018
[250]	train-mae:0.051988
[260]	train-mae:0.051958
[270]	train-mae:0.051929
[280]	train-mae:0.051901
[290]	train-mae:0.051874
[300]	train-mae:0.051844
[310]	train-mae:0.051818
[320]	train-mae:0.051791
[330]	train-mae:0.051767
[340]	train-mae:0.051745
[350]	train-mae:0.05172
[360]	train-mae:0.051695
[370]	train-mae:0.051671
[380]	train-mae:0.051648
[390]	train-mae:0.051624
[400]	train-mae:0.051